In [1]:
import pandas as pd

In [3]:
historical_stock_prices = pd.read_csv('daily-historical-stock-prices-1970-2018/historical_stock_prices.csv')

In [4]:
historical_stocks = pd.read_csv('daily-historical-stock-prices-1970-2018/historical_stocks.csv')

In [5]:
historical_stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [8]:
combined_stocks_df = pd.merge(historical_stock_prices, historical_stocks, on='ticker', how='inner')

In [21]:
combined_stocks_df.shape

(20973889, 12)

In [9]:
sorted_date = combined_stocks_df.sort_values(by='date', ascending=False)

In [10]:
sorted_date.head()

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
20973888,NZF,14.600000,14.690000,14.690000,14.590000,14.690000,180900,2018-08-24,NYSE,NUVEEN MUNICIPAL CREDIT INCOME FUND,NaN,NaN
5887096,OMED,2.380000,2.150000,2.150000,2.120000,2.410000,185300,2018-08-24,NASDAQ,"ONCOMED PHARMACEUTICALS, INC.",HEALTH CARE,MAJOR PHARMACEUTICALS
14268785,KMDA,5.600000,5.650000,5.650000,5.600000,5.710000,19600,2018-08-24,NASDAQ,KAMADA LTD.,HEALTH CARE,MAJOR PHARMACEUTICALS
5858822,AXP,105.910004,105.760002,105.760002,105.449997,106.339996,1844000,2018-08-24,NYSE,AMERICAN EXPRESS COMPANY,FINANCE,FINANCE: CONSUMER SERVICES
9914620,ILPT,23.980000,23.990000,23.990000,23.915001,24.041000,29000,2018-08-24,NASDAQ,INDUSTRIAL LOGISTICS PROPERTIES TRUST,CONSUMER SERVICES,REAL ESTATE INVESTMENT TRUSTS


In [11]:
grouped_stocks_ticker = combined_stocks_df.groupby('ticker')

In [12]:
max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']

In [39]:
max_date_df

1335          AHH
3200          APO
6144          PEZ
7300         CRCM
11321         GTN
16119        FLWS
19368        GHDX
25582        VIAV
30942         MHD
39890       AMSWA
49397        AAPL
58477        RAVN
59488        ISTR
63693        BRFS
70156         RRC
75479        EGHT
80094        CLWT
82364         GTT
87156        CCNE
87766        DSKE
89667        OXLC
93119         PEY
100619        MHF
103317        GTS
107023        AHL
110767       CAAS
120463        RRD
128010       AFSI
133686       SASR
143415        GTY
            ...  
20820533    GBLIZ
20821055     MDGS
20823197      DMO
20829583      NXN
20834444       GS
20841060      NXQ
20853334       GT
20859994      NXP
20865786       HA
20872345      NXR
20872743     JELD
20881621      DNB
20883038    FWONA
20884081    FWONK
20892439     SPGI
20892756     ARGX
20902069       HD
20902340     RMBL
20910249     MNST
20919945       HE
20931347      NYT
20936424      DNI
20942541     ISIG
20945452       HF
20949023  

In [13]:
currently_trading_ticker = max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']['ticker']

In [14]:
currently_trading_ticker = currently_trading_ticker.values

In [15]:
currently_trading_df = combined_stocks_df[combined_stocks_df['ticker'].isin(currently_trading_ticker)]

In [17]:
currently_trading_df

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
5,AHH,11.60,11.60,8.507822,11.54,11.60,76800,2013-05-15,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
6,AHH,11.62,11.74,8.610501,11.54,11.74,170300,2013-05-16,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
7,AHH,11.70,11.76,8.625172,11.70,11.85,305400,2013-05-17,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
8,AHH,11.76,11.73,8.603169,11.63,11.83,46800,2013-05-20,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
9,AHH,11.76,11.83,8.676512,11.61,11.84,77000,2013-05-21,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE


In [22]:
group_ticker = currently_trading_df.groupby('ticker')

In [25]:
max_trade_date = group_ticker['date'].max().values

In [29]:
cleaned_currently_trading_df = currently_trading_df.dropna(axis=0)

In [41]:
cleaned_currently_trading_df

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
5,AHH,11.60,11.60,8.507822,11.54,11.60,76800,2013-05-15,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
6,AHH,11.62,11.74,8.610501,11.54,11.74,170300,2013-05-16,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
7,AHH,11.70,11.76,8.625172,11.70,11.85,305400,2013-05-17,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
8,AHH,11.76,11.73,8.603169,11.63,11.83,46800,2013-05-20,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
9,AHH,11.76,11.83,8.676512,11.61,11.84,77000,2013-05-21,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE


In [33]:
grouped_2000_stocks_ticker = combined_stocks_df.groupby('ticker')
#max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']
date_2000_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2000-01-03']

In [45]:
sorted_current_trading = cleaned_currently_trading_df.sort_values(['date','ticker'])

In [46]:
sorted_current_trading = sorted_current_trading.reset_index()

In [48]:
sorted_current_trading.head()

,index,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
0,14218004,AA,7.140915,7.140915,1.986261,7.140915,7.225020,22500,1970-01-02,NYSE,ALCOA CORPORATION,BASIC INDUSTRIES,ALUMINUM
1,9736426,AEP,30.125000,30.625000,0.983416,30.125000,31.000000,10300,1970-01-02,NYSE,"AMERICAN ELECTRIC POWER COMPANY, INC.",PUBLIC UTILITIES,ELECTRIC UTILITIES: CENTRAL
2,15855608,ARNC,6.699775,6.699775,1.845695,6.699775,6.770052,24100,1970-01-02,NYSE,ARCONIC INC.,CAPITAL GOODS,METAL FABRICATIONS
3,15427303,BA,0.925926,0.979424,0.229325,0.925926,0.979424,634400,1970-01-02,NYSE,BOEING COMPANY (THE),CAPITAL GOODS,AEROSPACE
4,12465494,CAT,3.541667,3.458333,0.378521,3.447917,3.541667,276000,1970-01-02,NYSE,"CATERPILLAR, INC.",CAPITAL GOODS,CONSTRUCTION/AG EQUIPMENT/TRUCKS


In [53]:
#df.drop(df.index[3:5])
df_cleaned_2000 = sorted_current_trading.drop(sorted_current_trading.index[:4649822])